In [13]:
!pip install cdqa


Processing /igz/.cache/pip/wheels/11/59/5d/dcc580e12dccb97b5dcbb02a6ab9ea343dca9755b5ed1b5a2b/tika-1.24-py3-none-any.whl
  Attempting uninstall: tika
    Found existing installation: tika 1.24
    Uninstalling tika-1.24:
      Successfully uninstalled tika-1.24
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

cdqa 1.3.9 requires tika==1.19, but you'll have tika 1.24 which is incompatible.


In [1]:
import os
import pandas as pd
from ast import literal_eval


from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model


/User/.pythonlibs/jupyter-nb/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [28]:
download_model(model='bert-squad_1.1', dir='./models')

In [29]:
df = pd.read_csv('t2.csv', converters={'paragraphs': literal_eval})
#pd.set_option('display.max_colwidth', -1)


In [30]:
pd.set_option('display.max_colwidth', -1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
id            5 non-null int64
title         5 non-null object
url           5 non-null object
paragraphs    5 non-null object
dtypes: int64(1), object(3)
memory usage: 288.0+ bytes


In [31]:
#df.head()

In [32]:
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )
df.columns

Index(['id', 'title', 'url', 'paragraphs'], dtype='object')

In [33]:
import numpy as np
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [34]:
import joblib
joblib.dump(cdqa_pipeline, './models/bert_qa_custom.joblib')

['./models/bert_qa_custom.joblib']

In [35]:
cdqa_pipeline=joblib.load('./models/bert_qa_custom.joblib')
cdqa_pipeline

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [38]:
query = 'what are diabetes risks'
prediction = cdqa_pipeline.predict(query,5)

In [39]:
prediction

[('Thromboembolic risk',
  's41574-020-00435-4',
  'Thromboembolic risk in patients with diabetes melli-tus. Several publications have reported an increased thromboembolic risk among patients with diabetes mellitus outside the specific situation of SARS- CoV-2 infection. For example, a population- based study found that patients with T2DM exhibited an increased risk of venous thromboembolism compared with controls (HR 1.44, 95% CI 1.27–1.63)153. Furthermore, the risks of pulmonary embolism were greater in the patients with T2DM than in the controls (HR 1.52, 95% CI 1.22–1.90)153. Another study found that the incidence ',
  4.520924549177537),
 ('Preadmission diabetes- specific risk factors for mortality in hospitalized patients with diabetes and coronavirus disease',
  's41574-020-00435-4',
  '213. Agarwal, S., Schechter, C., Southern, W., Crandall, J. P. & Tomer, Y. Preadmission diabetes- specific risk factors for mortality in hospitalized patients with diabetes  and coronavirus disea

In [40]:
from cloudpickle import dumps


joblib.dump(cdqa_pipeline, 'bertqa.pkl')

['bertqa.pkl']

In [31]:
context.log_model("model",
                        body = dumps(cdqa_pipeline),
                        model_file='bert_qa.pkl',
                        labels={"class": "QAPipeline"})
    

NameError: name 'context' is not defined